<a href="https://colab.research.google.com/github/Termote/Deep_learning_for_optical_imaging/blob/main/Final_project_DefeatChatGPT_opti_WITZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Data

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Input, Activation
from keras.layers import Conv2D, MaxPooling2D,TimeDistributed, Concatenate, Conv3D, MaxPooling3D, BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split

import cv2
import os
import gdown



In [2]:
# Download the data training folder
url = "https://drive.google.com/drive/folders/1ud-8x1rVE2pG2Tnh40okr33t47QfKNAc"
gdown.download_folder(url, quiet=True, use_cookies=False)



Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1zSda57x2TtlC0h-7VTOL18bpnQEn715y 



In [3]:
import pandas as pd
import os

# File paths and corresponding column names
file_columns = [('/content/Project Files/Training Data/common_time.npy', 'common_time'),
                ('/content/Project Files/Training Data/ground_truth.npy', 'ground_truth'),
                ('/content/Project Files/Training Data/labels.npy', 'labels'),
                ('/content/Project Files/Training Data/meteo_data.csv', 'meteo_data')]
# Create an empty DataFrame
combined_df = pd.DataFrame()

# Load and add data to the DataFrame
for file_path, column_name in file_columns:
    if file_path.endswith('.npy'):
        data = np.load(file_path, allow_pickle=True)
        df = pd.DataFrame(data, columns=[column_name])
        combined_df = pd.concat([combined_df, df], axis=1)
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
        df.columns = [f"{column_name}_{col}" for col in df.columns]
        combined_df = pd.concat([combined_df, df], axis=1)

# Display the combined DataFrame
print("Combined DataFrame:")

#remove one of the date columns
combined_df = combined_df.drop('meteo_data_time', axis=1)
print(combined_df.shape)
combined_df.head(5)

FileNotFoundError: ignored

In [ ]:
# Load images
x1 = np.load('/content/Project Files/Training Data/X1.npy')
x2 = np.load('/content/Project Files/Training Data/X2.npy')


In [ ]:
# Create a subplot with two rows and five columns
fig, axes = plt.subplots(2, 5, figsize=(15, 5))

# Display images in the subplot
for i in range(5):
    axes[0, i].imshow(x1[i])
    axes[0, i].axis('off')
    axes[1, i].imshow(x2[i])
    axes[1, i].axis('off')

plt.show()

print("x1 shape: ", x1.shape)
print("x2 shape: ", x2.shape)

In [ ]:
# Resizing the images to new dimensions

# take only every two images and combined_df
x1 = x1[::4]
x2 = x2[::4]
combined_df = combined_df.iloc[::4]

new_dim = 100
image_crop_percent = 0 # (top, bottom, left, right)
side_dim_scaled = (new_dim, int(new_dim * (1 - image_crop_percent)))

x1_resized = np.array([cv2.resize(img, (new_dim, new_dim)) for img in x1])
del x1
# crop the images
x1_resized = x1_resized[:, 0:int(x1_resized.shape[1]*(1-image_crop_percent))]

x2_resized = np.array([cv2.resize(img, (new_dim, new_dim)) for img in x2])
del x2
# crop the images
x2_resized = x2_resized[:, 0:int(x2_resized.shape[1]*(1-image_crop_percent))]

# normalize the images

x1_resized = x1_resized / 255.0
x2_resized = x2_resized / 255.0

new_dim = (x1_resized.shape[1], x1_resized.shape[2])

In [ ]:
print("x1_resized shape: ", x1_resized.shape)
print("x2_resized shape: ", x2_resized.shape)

# Create a subplot with two rows and five columns
fig, axes = plt.subplots(2, 5, figsize=(10, 5))

# Display images in the subplot
for i in range(5):
    axes[0, i].imshow(x1_resized[i])
    axes[0, i].axis('off')
    axes[1, i].imshow(x2_resized[i])
    axes[1, i].axis('off')

plt.show()

In [ ]:
#Split data

def date_to_cyclic(day, month):
    c_day = np.cos(day * (2 * np.pi / 31))
    s_day = np.sin(day * (2 * np.pi / 31))
    c_month = np.cos(month * (2 * np.pi / 12))
    s_month = np.sin(month * (2 * np.pi / 12))
    return c_day, s_day, c_month, s_month

def split_train_test(data, train_test_ratio):
    train_size = int(len(data) * train_test_ratio)
    return data[:train_size], data[train_size:]

def create_train_test(train_test_ratio):

    date = combined_df['common_time']
    # convert to datetime
    date = pd.to_datetime(date)
    # extract the month
    month = date.dt.month
    # extract the day
    day = date.dt.day

    # convert to cyclic
    c_day, s_day, c_month, s_month = date_to_cyclic(day, month)
    
    columns = ['meteo_data_Wind_speed', 'meteo_data_Wind_dir', 'meteo_data_Air_temp','ground_truth']
    meteo_data = combined_df[columns]
    meteo_data = np.array(meteo_data)

    # Add cyclic date to meteo_data
    meteo_data = np.concatenate((meteo_data, c_day[:, None], s_day[:, None], c_month[:, None], s_month[:, None]), axis=1)

    labels = combined_df['labels']
    labels = np.array(labels)

    train_x1, test_x1 = split_train_test(x1_resized, train_test_ratio)
    train_x2, test_x2 = split_train_test(x2_resized, train_test_ratio)

    train_data, test_data = split_train_test(meteo_data, train_test_ratio)

    y_train, y_test = split_train_test(labels, train_test_ratio)

    X_images_train = np.stack((train_x1, train_x2), 1)
    X_images_test = np.stack((test_x1, test_x2), 1)

    # Shuffle the data for training
    idx = np.random.permutation(len(X_images_train))

    X_images_train, X_data_train, y_train = X_images_train[idx], train_data[idx], y_train[idx]

    return X_images_train, X_data_train, X_images_test, test_data, y_train, y_test


train_test_ratio = 0.9

X_images_train, X_data_train, X_images_test, X_data_test, y_train, y_test = create_train_test(train_test_ratio)

print("X_images_train shape:", X_images_train.shape)
print("X_data_train shape:", X_data_train.shape)
print("X_images_test shape:", X_images_test.shape)
print("X_data_test shape:", X_data_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

del x1_resized
del x2_resized

In [ ]:
# print the first 2 images with labels and corresponding wind speed

fig, axes = plt.subplots(2, 4, figsize=(10, 5))

# Display images in the subplot
for i in range(4):

    axes[0, i].imshow(X_images_train[i][0])
    axes[0, i].axis('off')
    # plot title as label and wind speed
    axes[0, i].set_title("Label: {} \nWind speed: {}\nWind direction: {}\nAir temp: {}\nGround Truth: {}".format(y_train[i], X_data_train[i,0],X_data_train[i,1],X_data_train[i,2],X_data_train[i,3]))
    axes[1, i].imshow(X_images_train[i][1])
    axes[1, i].axis('off')
    axes[1, i].set_title("Label: {} \nWind speed: {}\nWind direction: {}\nAir temp: {}\nGround Truth: {}".format(y_train[i], X_data_train[i,0],X_data_train[i,1],X_data_train[i,2],X_data_train[i,3]))

plt.tight_layout()
plt.show()


ChatGPT model


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LSTM, Dense, Flatten, concatenate
import numpy as np
# import exponentialdecay



In [ ]:
channels = 3

input_image = tf.keras.layers.Input(shape=(2,new_dim[0], new_dim[1], channels), name='image_input')

input_data = tf.keras.layers.Input(shape=(8,), name='weather_data_input')

# Hidden layers for data input
data_input = tf.keras.layers.Dense(8, activation='relu')(input_data)
data_input = tf.keras.layers.Dense(32, activation='relu')(input_data)
data_input = tf.keras.layers.Dense(8, activation='relu')(data_input)

# CNN Block 1
first_input = tf.keras.layers.TimeDistributed(Conv2D(8, kernel_size=3, activation='relu'))(input_image)
first_input = tf.keras.layers.TimeDistributed(MaxPooling2D(pool_size = 2))(first_input)
first_input = tf.keras.layers.TimeDistributed(Conv2D(8, kernel_size=3, activation='relu'))(first_input)
first_input = tf.keras.layers.TimeDistributed(MaxPooling2D(pool_size = 2))(first_input)

# CNN Block 2
first_input = tf.keras.layers.TimeDistributed(Conv2D(16, kernel_size=3, activation='relu'))(first_input)
first_input = tf.keras.layers.TimeDistributed(MaxPooling2D(pool_size = 2))(first_input)
first_input = tf.keras.layers.TimeDistributed(Conv2D(16, kernel_size=3, activation='relu'))(first_input)
first_input = tf.keras.layers.TimeDistributed(MaxPooling2D(pool_size = 2))(first_input)

# Flattening and Reshaping for LSTM
flt = tf.keras.layers.TimeDistributed(Flatten())(first_input)
flt = tf.keras.layers.Reshape((1, -1))(flt)

# Reshape data input for concatenation
data_input = tf.keras.layers.Reshape((1, -1))(data_input)

# Concatenating CNN and data input
concat = tf.keras.layers.concatenate([flt, data_input])

# LSTM
lstm1 = tf.keras.layers.LSTM(64, activation='relu', return_sequences=False)(concat)

# Output
dense1 = tf.keras.layers.Dense(1, activation='relu')(lstm1)

# Setting input and ouput of model
model = tf.keras.models.Model(inputs=[input_image, input_data], outputs=dense1)


In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

# compile the model and use mse as loss function, use as metric accuracy or mse itself 
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001, decay_steps=250, decay_rate=0.9
)

model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mean_squared_error')

print('Graphical schematic of the compiled network')
print()
model.summary()

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(filename='model.png')

In [ ]:
from sklearn.metrics import mean_squared_error

rmse_list = []

def reset_model(model):

    model_copy = tf.keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())

    return model_copy

for i in range(5):
    print('Training iteration: ', i+1)
    print()

    model_copy = reset_model(model)
    model_copy.compile(optimizer='adam', loss='mean_squared_error') # default lr = 0.001

    history = model_copy.fit([X_images_train, X_data_train], y_train, shuffle=True, epochs=50, batch_size=128, verbose = 2, validation_split=0.1)
    print()
    print('Training rmse: ', np.sqrt(history.history['loss'][-1]))
    print('Validation rmse: ', np.sqrt(history.history['val_loss'][-1]))

    y_pred = model_copy.predict([X_images_test, X_data_test])
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print('Test accuracy: ', rmse)
    print()
    rmse_list.append(rmse)
    if i != 4 :
      del model_copy
      del history

#Averaging the test accuracy after 5 times training
print('Average test rmse: ', np.mean(rmse_list))
print('Standard deviation of test rmse: ', np.std(rmse_list))

In [ ]:
# Plot the training and validation loss

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
# Plot the RMSE loss

plt.plot(np.sqrt(history.history['loss']), label='RMSE Training loss')
plt.plot(np.sqrt(history.history['val_loss']), label='RMSE Validation loss')
plt.legend()
plt.show()


In [ ]:
# Evaluate the model

from sklearn.metrics import mean_squared_error

y_pred = model_copy.predict([X_images_test, X_data_test])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("RMSE:", rmse)
